# SIM データの比較・決定係数の算出

04. <u>**実験, SIM　data の比較・決定係数の算出**</u>

    - `状況　:　新規の科学反応チャンバー開発をおこなており、シュミレーションを活用したいと考えている`
    
        - **データ分析の目的**
            - *シュミレーションモデルが正しく設計できているか、実験とシュミレーションの結果を比較する*
        
        - **保有している実験データ**
            - *反応チャンバーでの化学種 A, B, C, D の時系列変化の3回分の実験 data と設計した SIM model の　data*
            
        - **起こっている化学反応**
            - *A + B + 2C*
            - *A + C -> D*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
plt.style.use('ggplot')

In [3]:
df_sim = pd.read_csv('./resources/SIM(Reaction)_1.csv', delimiter=',', header=9, index_col=0)
df_sim2 = pd.read_csv('./resources/SIM(Reaction)_2.csv', delimiter=',', header=9, index_col=0)

In [4]:
df_sim.head()

,A,B,C,D
Time[min],,,,
0,1.000000,1.000000,0.000000,0.000000
5,0.959854,0.960018,0.039819,0.000164
10,0.922501,0.923121,0.076259,0.000620
15,0.887656,0.888978,0.109700,0.001322
20,0.855069,0.857300,0.140470,0.002230


In [5]:
df_sim2.head()

,A,B,C,D
Time[min],,,,
450,0.135126,0.250201,0.634724,0.115075
455,0.133024,0.248808,0.635408,0.115784
460,0.130961,0.247444,0.636073,0.116483
465,0.128936,0.246108,0.636720,0.117172
470,0.126950,0.244800,0.637349,0.117851


In [7]:
# DataFrame を merge する
df_sim3 = pd.concat([df_sim, df_sim2])
df_sim3

,A,B,C,D
Time[min],,,,
0,1.000000,1.000000,0.000000,0.000000
5,0.959854,0.960018,0.039819,0.000164
10,0.922501,0.923121,0.076259,0.000620
15,0.887656,0.888978,0.109700,0.001322
20,0.855069,0.857300,0.140470,0.002230
...,...,...,...,...
980,0.030309,0.184767,0.660774,0.154458
985,0.029912,0.184536,0.660840,0.154624
990,0.029520,0.184308,0.660905,0.154788


- **同じく実験 data も読み込んで繋げていく**

In [8]:
# 1. 空の DataFrame を作成
df_ex = pd.DataFrame([])

# 2. excel から　for loop で sheet を順番に読み込む
for i in range(1, 4):
    df = pd.read_excel('./resources/exp(Reaction).xlsx', sheet_name='Sheet'+str(i), header=5, index_col=0)
    # 3. 新たな　col を作成して i を代入 : 実験が何回目かの区別する為
    df['id'] = i
    # 4. 空のDataFrame に　　merge
    df_ex = pd.concat([df_ex, df])

In [9]:
df_ex

,A(exp),B(exp),C(exp),D(exp),id
Time[min],,,,,
0,1.000000,1.000000,0.000000,0.000000,1
30,0.815000,0.770000,0.223000,0.006000,1
60,0.669000,0.659000,0.313000,0.060000,1
90,0.569000,0.556000,0.369000,0.068000,1
120,0.512000,0.500000,0.437000,0.027000,1
...,...,...,...,...,...
870,0.078779,0.272384,0.581398,0.157232,3
900,0.108853,0.180126,0.555623,0.223785,3
930,0.079660,0.225168,0.682219,0.159969,3


- **横軸を時間・縦軸をそれぞれの化学集の濃度として, A, B, C, D を plot する**

In [35]:
# id で group にする
groups = df_ex.groupby('id')

# 実験の回数目ごとに　subplotで分けて描画させる
plt.figure(figsize=(15, 10))

# A B C D を plot させる為の list
species = ['A', 'B', 'C', 'D']

for name, group in groups:
    plt.subplot(2, 2, name)
    for i in species:
        plt.scatter(group.index, group[i + '(exp)'], label=str(i)+ ' exp')
        plt.plot(df_sim3.index, df_sim3[i], label=str(i)+ ' sim')
        plt.title(str(name)+' time experiment')
    plt.xlabel('Time[min]')
    plt.legend(bbox_to_anchor=(1.2,1.0))
    plt.tight_layout()

# **決定係数の算出**

- **実線　=　simulation data**
    
    - シュミレーションデータ


- **点　=　experiment data**
    
    - 実験データ


- *上記を見比べると比較的会っていそうだと思われるが...*

    - **決定係数で評価する**　:　定量的に評価する尺度
> https://ja.wikipedia.org/wiki/%E6%B1%BA%E5%AE%9A%E4%BF%82%E6%95%B0


### <u>決定係数</u>
### $$
R^2 =1 - \displaystyle\frac{\sum_{i}{\left(y_i - f_i \right)}^2}{\sum_{i}{\left(y_i - \overline{y} \right)}^2}
$$



### <u>分母</u>
- **$y_i$**　ー　**$\overline{y}$**　=　**分散**$^2$

- **$y_i$**:*実験データ*　ー　**$\overline{y}$**:*実験平均データ*　＝　**分散**$^2$


### <u>分子</u>
- **$y_i$**　ー　**$f_i$**　=　**実験データ - sim = 差分**$^2$

- **$y_i$**:*実験データ*　ー　**$f_i$**:*simulationデータ*　＝　**差分**$^2$


### <u>sigma</u>
- **$\sum_{i}$**　＝　全ての総和

### バラつきを抑える為に3回の実験の平均値を算出
- **時間毎に groupby をする**

In [36]:
group_t = df_ex.groupby(level='Time[min]')

In [37]:
# A,B,C,D の化学集にアクセスする -> 実験１,2,3を足して平均値を取る -> A,B,C,D の時間毎の3回の平均値を取得できる
species_ex = ['A(exp)', 'B(exp)', 'C(exp)', 'D(exp)']
df_ave = group_t[species_ex].mean()

In [38]:
# df_ex ： 平均値
df_ave.head()

,A(exp),B(exp),C(exp),D(exp)
Time[min],,,,
0,1.000000,1.000000,0.000000,0.000000
30,0.818080,0.769971,0.205880,0.031531
60,0.641198,0.644452,0.269237,0.068027
90,0.589785,0.576097,0.370085,0.053969
120,0.513866,0.525938,0.430062,0.044208


### <u>分母</u>
- **$y_i$**　ー　**$\overline{y}$**　=　**分散**$^2$
- **$y_i$**:*実験データ*　ー　**$f_i$**:*simulationデータ*　＝　**差分**$^2$

In [39]:
# 実験データとシュミレーションデータの差分をとる
df_errors = pd.DataFrame()
for i in species:
    diff = (df_ave[i+'(exp)'] - df_sim3[i]) ** 2
    df_errors[i] = diff

In [43]:
# NaN を削除 / 30分毎の差分データがある所だけを取得できる
df_errors = df_errors.dropna()
df_errors

,A,B,C,D
Time[min],,,,
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
30,4.949457e-04,9.243204e-04,1.165246e-04,7.284483e-04
60,1.768969e-04,5.719340e-04,2.354770e-03,2.932989e-03
90,1.534399e-03,6.531124e-07,8.971075e-04,8.581613e-04
120,1.842081e-03,3.755786e-04,7.711147e-04,7.389040e-05
150,4.758294e-04,2.130975e-05,3.877152e-04,2.304434e-03
180,9.097688e-05,7.257268e-04,1.704501e-03,4.555006e-04
210,1.444935e-05,9.046553e-05,4.030392e-04,3.018810e-07
240,9.538875e-06,9.349922e-04,8.188934e-05,4.884413e-06


### <u>sigma</u>
- **$\sum_{i}$**　＝　全ての総和

In [44]:
# 分子
df_num = df_errors.sum()
df_num.head()

A    0.033022
B    0.025805
C    0.021283
D    0.036542
dtype: float64

In [46]:
# 分母　: 分散を計算することができる / ddof=0 通常の分散 * dataの数
denom = df_ave.var(ddof=0) * df_ave.shape[0]
denom

A(exp)    1.818340
B(exp)    1.205078
C(exp)    0.778009
D(exp)    0.094567
dtype: float64

In [47]:
for i, j in enumerate(species):
    print(j, 1.0 - df_num[j] / denom[j+'(exp)'])

A 0.9818392277242951
B 0.9785867487484551
C 0.9726437013080427
D 0.6135868384449772
